In [1]:
import franky

In [3]:
robot = franky.Robot("172.168.0.2", realtime_config=franky.RealtimeConfig.Ignore)
robot.relative_dynamics_factor = 0.2

In [ ]:
robot.recover_from_errors()

In [ ]:
robot.current_joint_state.position

In [6]:
home_pos = [0.0, -0.31, 0.0, -1.53, 0.0, 1.522, 0.785]

In [13]:
motion = franky.JointMotion(home_pos)

In [14]:
robot.move(motion, asynchronous=False)

In [ ]:
robot.current_pose.end_effector_pose

In [ ]:
motion = CartesianMotion(Affine([-0.2, 0.0, 0.0]), ReferenceType.Relative)

In [ ]:
motion = CartesianMotion(
    Affine([0.45, 0, 0.55], [1, 0, 0, 0]), ReferenceType.Absolute)

In [ ]:
motion = JointWaypointMotion([
    JointWaypoint([0.0,  -0.3, 0.0, -1.8, 0.0, 1.5,  0.65])])

In [ ]:
robot.move(motion, asynchronous=True)

In [ ]:
gripper = franky.Gripper("172.168.0.2")

In [ ]:
gripper.open(0.02)

In [ ]:
import numpy as np

In [ ]:
vel = 0.03
gripper.open(vel)
try:
    fut = None
    for w in np.linspace(0, 100, 100 * 10):
        if fut is not None and not fut.wait(0):
            gripper.terminate_command()
        fut = gripper.move_async(w * gripper.state.max_width, vel)
except KeyboardInterrupt:
    gripper.stop()

--------
# Now starts Kinova

In [11]:
import collections
collections.MutableMapping = collections.abc.MutableMapping
collections.MutableSequence = collections.abc.MutableSequence
collections.MutableSet = collections.abc.MutableSet

In [12]:
import collections
import time
import threading

In [13]:
from kortex_api.autogen.client_stubs.ActuatorConfigClientRpc import ActuatorConfigClient
from kortex_api.autogen.client_stubs.BaseClientRpc import BaseClient
from kortex_api.autogen.client_stubs.BaseCyclicClientRpc import BaseCyclicClient
from kortex_api.autogen.messages import Base_pb2, BaseCyclic_pb2, ActuatorConfig_pb2

from kortex_api.TCPTransport import TCPTransport
from kortex_api.UDPTransport import UDPTransport

from kortex_api.RouterClient import RouterClient, RouterClientSendOptions
from kortex_api.SessionManager import SessionManager
from kortex_api.autogen.messages import Session_pb2

In [14]:
import geom
import autograd.numpy as np
import ruckig
import matplotlib.pyplot as plt


## Low level control

In [15]:
target_pos = geom.Transform3D(
    translation=np.array([0.53686397, 0.07407813, 0.53547285]),
    quaternion=np.array([0.50436832, 0.51220832, 0.48933736, 0.49376531]))

In [16]:
def inverse_kinematics(base, transform, joints_guess):
    euler = geom.radians_to_degrees(geom.quat_to_euler(transform.quaternion))
    ik_data = Base_pb2.IKData()
    ik_data.cartesian_pose.x = transform.translation[0]
    ik_data.cartesian_pose.y = transform.translation[1]
    ik_data.cartesian_pose.z = transform.translation[2]
    ik_data.cartesian_pose.theta_x = euler[0]
    ik_data.cartesian_pose.theta_y = euler[1]
    ik_data.cartesian_pose.theta_z = euler[2]

    for a in joints_guess:
        ja = ik_data.guess.joint_angles.add()
        ja.value = a

    joint_angels = base.ComputeInverseKinematics(ik_data)
    return np.array([a.value for a in joint_angels.joint_angles])

In [17]:
def normalise_angles(target, base, period=360):
    res = target - base
    res[res > period / 2] -= period
    res[res < -period / 2] += period
    return base + res

In [8]:
class KinovaController:
    def __init__(self, ip: str):
        self.stop_event = threading.Event()
        self.target_pos = None
        self.target_lock = threading.Lock()

        self.output_lock = threading.Lock()
        self.output = None

    def run(self):
        if True:
            transport = TCPTransport()
            router = RouterClient(transport, RouterClient.basicErrorCallback)
            transport.connect('192.168.1.10', 10000)

            session_info = Session_pb2.CreateSessionInfo()
            session_info.username = "admin"
            session_info.password = "admin"
            session_info.session_inactivity_timeout = 1000 * 1000   # (milliseconds)
            session_info.connection_inactivity_timeout = 1000 * 1000 # (milliseconds)

            sessionManager = SessionManager(router)
            sessionManager.CreateSession(session_info)

            base = BaseClient(router)
            base.SetServoingMode(Base_pb2.ServoingModeInformation(servoing_mode = Base_pb2.SINGLE_LEVEL_SERVOING))

            udp_transport = UDPTransport()
            upd_router = RouterClient(udp_transport, RouterClient.basicErrorCallback)
            udp_transport.connect('192.168.1.10', 10001)

            udp_session = SessionManager(upd_router)
            udp_session.CreateSession(session_info)
            base_cyclic = BaseCyclicClient(upd_router)

            action_list = base.ReadAllActions(
                Base_pb2.RequestedActionType(action_type = Base_pb2.REACH_JOINT_ANGLES))
            action_handle = [a.handle for a in action_list.action_list
                            if a.name == "Home"][0]

            def check(n, e):
                if n.action_event in [Base_pb2.ACTION_END, Base_pb2.ACTION_ABORT]:
                    print("Home position reached")
                    e.set()

            e = threading.Event()
            notification_handle = base.OnNotificationActionTopic(lambda n: check(n, e), Base_pb2.NotificationOptions())

            base.ExecuteActionFromReference(action_handle)
            finished = e.wait(30)
            if not finished:
                raise Exception("Home position not reached")
            base.Unsubscribe(notification_handle)
        # ------------------


        control = ruckig.Ruckig(7, 1 / 1000)  # 40 hz
        inp_ctrl = ruckig.InputParameter(7)

        inp_ctrl.target_velocity = [0.0] * 7
        inp_ctrl.target_acceleration = [0.0] * 7

        inp_ctrl.max_position = [np.inf, 128.9, np.inf, 147.8, np.inf, 120.3, np.inf]
        inp_ctrl.min_position = [-np.inf, -128.9, -np.inf, -147.8, -np.inf, -120.3, -np.inf]
        inp_ctrl.max_velocity = [20.0] * 7
        inp_ctrl.max_acceleration = [15.0] * 7
        inp_ctrl.max_jerk = [6.0] * 7

        target_joints = None
        command = BaseCyclic_pb2.Command()
        for i in range(7):
            act = command.actuators.add()

        send_option = RouterClientSendOptions()
        send_option.timeout_ms = 50

        feedback = base_cyclic.RefreshFeedback()
        inp_ctrl.current_position = ([a.position for a in feedback.actuators])
        inp_ctrl.current_velocity = ([a.velocity for a in feedback.actuators])
        with self.output_lock:
            self.output = np.array([a.position for a in feedback.actuators])
        out = ruckig.OutputParameter(7)
        res = ruckig.Result.Working
        start = time.monotonic()
        count = 0

        base.SetServoingMode(Base_pb2.ServoingModeInformation(servoing_mode=Base_pb2.LOW_LEVEL_SERVOING))
        try:
            while not self.stop_event.is_set():
                with self.target_lock:
                    if self.target_pos is not None:
                        if target_joints is None:
                            start = time.monotonic()

                        target_pos = self.target_pos
                        target_joints = inverse_kinematics(base, target_pos, [a.position for a in feedback.actuators])
                        self.target_pos = None
                        inp_ctrl.target_position = normalise_angles(target_joints, inp_ctrl.current_position)

                if target_joints is None:
                    time.sleep(0.1)
                    continue

                inp_ctrl.target_position = normalise_angles(target_joints, inp_ctrl.current_position)

                res = control.update(inp_ctrl, out)
                out.pass_to_input(inp_ctrl)

                command.frame_id = (command.frame_id + 1) % 65536
                for i in range(7):
                    command.actuators[i].command_id = command.frame_id
                    # command.actuators[i].position = feedback.actuators[i].position
                    command.actuators[i].position = out.new_position[i]

                feedback = base_cyclic.Refresh(command, 0, send_option)
                with self.output_lock:
                    self.output = np.array([a.position for a in feedback.actuators])

                count += 1
                if count % 10000 == 9999:
                    print(f'Running {count / (time.monotonic() - start):.2f} hz')

        finally:
            base.SetServoingMode(Base_pb2.ServoingModeInformation(servoing_mode=Base_pb2.SINGLE_LEVEL_SERVOING))
            base.Stop()

            # if sessionManager != None:
            #     router_options = RouterClientSendOptions()
            #     router_options.timeout_ms = 1000
            #     sessionManager.CloseSession(router_options)

            # udp_transport.disconnect()
            # transport.disconnect()

In [ ]:
controller = KinovaController("192.168.1.10")
control_thread = threading.Thread(target=controller.run)
control_thread.start()

In [19]:
transport = TCPTransport()
router = RouterClient(transport, RouterClient.basicErrorCallback)
transport.connect('192.168.1.10', 10000)

session_info = Session_pb2.CreateSessionInfo()
session_info.username = "admin"
session_info.password = "admin"
session_info.session_inactivity_timeout = 1000 * 1000   # (milliseconds)
session_info.connection_inactivity_timeout = 1000 * 1000 # (milliseconds)

sessionManager = SessionManager(router)
sessionManager.CreateSession(session_info)

base = BaseClient(router)
cyclic = BaseCyclicClient(router)

In [ ]:
cyclic.RefreshFeedback()

In [13]:
from hardware import kinova as knv

In [ ]:
base.SetServoingMode(Base_pb2.ServoingModeInformation(servoing_mode = Base_pb2.SINGLE_LEVEL_SERVOING))
angles_pb = base.GetMeasuredJointAngles()

In [ ]:
for i in range(7):
    angles_pb.joint_angles[i].value = 0

In [ ]:
fk_out = base.ComputeForwardKinematics(angles_pb)
fk_out

In [ ]:
angles = np.array([a.value for a in angles_pb.joint_angles])

In [7]:
def robot_fk(angles):
    angles = geom.radians_to_degrees(angles)
    angles_pb = Base_pb2.JointAngles()
    for i, angle in enumerate(angles):
        angles_pb.joint_angles.add(joint_identifier=i, value=angle)
    fk_out = base.ComputeForwardKinematics(angles_pb)
    return geom.Transform3D(translation=np.array([fk_out.x, fk_out.y, fk_out.z]),
                            quaternion=geom.Quaternion.from_euler([fk_out.theta_x, fk_out.theta_y, fk_out.theta_z]))

In [8]:
def kinematic_model(params, angles):
    PARAMS_PER_LINK = 6
    result = geom.Transform3D()
    for i in range(7):
        start_offset = i * PARAMS_PER_LINK
        t = geom.Transform3D(translation=params[start_offset: start_offset + 3],
                             quaternion=geom.Quaternion.from_euler(params[start_offset + 3: start_offset + 6]))
        result = result * t * geom.Transform3D(quaternion=geom.Quaternion.from_euler([0, 0, angles[i]]))

    t_last = geom.Transform3D(translation=params[-6:-3],
                              quaternion=geom.Quaternion.from_euler(params[-3:]))
    return result * t_last

In [19]:
import concurrent.futures
AW = 1

def objective_function(params, joint_angels, target):
    fk = kinematic_model(params, joint_angels)
    diff = fk.translation - target.translation
    diff_norm = diff.dot(diff)
    return diff_norm + AW * (fk.quaternion.inv * target.quaternion).angle

def parallel_objective_function(params):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(objective_function, params, angles, target)
                   for angles, target in zip(joint_angles_set, target_set)]
        results = [f.result() for f in futures]
    return sum(results)

In [20]:
max_position = geom.degrees_to_radians(np.array([180, 128.9, 180, 147.8, 180, 120.3, 180]))

In [21]:
joint_angles_set = np.random.uniform(-max_position, max_position, size=(30, 7))
target_set = [robot_fk(angles) for angles in joint_angles_set]

In [17]:
init_params = np.zeros(8 * 6)
# for i, t in enumerate(knv._KINOVA_CHAIN):
#     init_params[i * 6: (i + 1) * 6] = np.concatenate([t.translation, t.quaternion.as_euler])

In [22]:
from autograd import grad
objective_function_grad = grad(parallel_objective_function)

In [ ]:
from scipy.optimize import minimize
count = 0

def callback(params):
    global count
    count += 1
    if count % 3 == 0:
        total_error = sum(objective_function(params, joint_angles_set[i], target_set[i])
                        for i in range(len(joint_angles_set)))
        print(f"Current parameters: {params}")
        print(f"Current error: {total_error}")

result = minimize(
    lambda params: sum(objective_function(params, joint_angles_set[i], target_set[i])
                        for i in range(len(joint_angles_set))),
    init_params,
    jac=objective_function_grad,
    method='BFGS', callback=callback, options={'disp': True})

In [ ]:
robot_fk(angles)

In [ ]:
def forward_kinematics(joints):
    joints = geom.degrees_to_radians(joints)

    result = geom.Transform3D()
    for chain, joint in zip(knv._KINOVA_CHAIN, joints):
        result = result * chain * geom.Transform3D(quaternion=geom.Quaternion.from_euler([0, 0, joint]))
    result = result * knv._KINOVA_CHAIN[-1]

    return result

In [ ]:
geom.radians_to_degrees(knv._forward_kinematics(angles).quaternion.as_euler)

In [ ]:
with controller.target_lock:
    controller.target_pos = target_pos

In [ ]:
target_pos2 = geom.Transform3D(translation=np.array([0.5, -0.15, 0.4]), quaternion=np.array([0.5, 0.5, 0.5, 0.5]))
with controller.target_lock:
    controller.target_pos = target_pos2

In [ ]:
controller.stop_event.set()
control_thread.join()

del controller

---

In [ ]:
log = collections.defaultdict(list)
actuator_mode = [ActuatorConfig_pb2.POSITION] * 7

base.SetServoingMode(Base_pb2.ServoingModeInformation(servoing_mode=Base_pb2.SINGLE_LEVEL_SERVOING))
try:
    feedback = base_cyclic.RefreshFeedback()
    out = ruckig.OutputParameter(7)
    res = ruckig.Result.Working
    start = time.monotonic()
    count = 0
    while res == ruckig.Result.Working:
        cur_pos = np.array([a.position for a in feedback.actuators])
        cur_vel = np.array([a.velocity for a in feedback.actuators])
        inp_ctrl.current_position = cur_pos
        # inp_ctrl.current_velocity = cur_vel
        inp_ctrl.target_position = normalise_angles(target_joints, inp_ctrl.current_position)

        res = control.update(inp_ctrl, out)
        log['pos/current'].append(normalise_angles(cur_pos, inp_ctrl.current_position))
        log['pos/target'].append(inp_ctrl.target_position)
        log['vel/current'].append(cur_vel)
        log['pos/output'].append(out.new_position)
        log['vel/output'].append(out.new_velocity)
        out.pass_to_input(inp_ctrl)

        joint_speeds = Base_pb2.JointSpeeds()
        for i in range(7):
            js = joint_speeds.joint_speeds.add()
            js.joint_identifier = i
            js.value = out.new_velocity[i]
            js.duration = 0

        base.SendJointSpeedsCommand(joint_speeds)
        feedback = base_cyclic.RefreshFeedback()
        count += 1
        if count % 100 == 99:
            print(f'Running {count / (time.monotonic() - start):.2f} hz')
finally:
    base.Stop()

In [ ]:
base.ClearFaults()
base.ExecuteActionFromReference(action_handle)

In [ ]:
for k in log:
    log[k] = np.array(log[k])

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(15, 8))
for i in range(7):
    row = i // 2
    col = (i % 2) * 2
    axs[row, col].plot(log['pos/current'][:, i])
    axs[row, col].plot(log['pos/target'][:, i])
    axs[row, col].plot(log['pos/output'][:, i])
    axs[row, col].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}'))
    axs[row, col].tick_params(axis='x', labelbottom=False)

    axs[row, col + 1].plot(log['vel/output'][:, i])
    axs[row, col + 1].plot(log['vel/current'][:, i])
    axs[row, col + 1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}'))
    axs[row, col + 1].tick_params(axis='x', labelbottom=False)

# Hide the empty subplots
for j in range(7, 8):
    row = j // 2
    col = (j % 2) * 2
    fig.delaxes(axs[row, col])
    fig.delaxes(axs[row, col + 1])

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(15, 8))
for i in range(7):
    row = i // 2
    col = (i % 2) * 2
    axs[row, col].plot(log['pos/current'][:, i])
    axs[row, col].plot(log['pos/target'][:, i])
    axs[row, col].plot(log['pos/output'][:, i])
    axs[row, col].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}'))
    axs[row, col].tick_params(axis='x', labelbottom=False)

    axs[row, col + 1].plot(log['vel/output'][:, i])
    axs[row, col + 1].plot(log['vel/current'][:, i])
    axs[row, col + 1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}'))
    axs[row, col + 1].tick_params(axis='x', labelbottom=False)

# Hide the empty subplots
for j in range(7, 8):
    row = j // 2
    col = (j % 2) * 2
    fig.delaxes(axs[row, col])
    fig.delaxes(axs[row, col + 1])

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(15, 8))
for i in range(7):
    row = i // 2
    col = (i % 2) * 2
    axs[row, col].plot(log['pos/current'][:, i])
    axs[row, col].plot(log['pos/target'][:, i])
    axs[row, col].plot(log['pos/output'][:, i])
    axs[row, col].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}'))
    axs[row, col].tick_params(axis='x', labelbottom=False)

    axs[row, col + 1].plot(log['vel/output'][:, i])
    axs[row, col + 1].plot(log['vel/current'][:, i])
    axs[row, col + 1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}'))
    axs[row, col + 1].tick_params(axis='x', labelbottom=False)

# Hide the empty subplots
for j in range(7, 8):
    row = j // 2
    col = (j % 2) * 2
    fig.delaxes(axs[row, col])
    fig.delaxes(axs[row, col + 1])

plt.tight_layout()
plt.show()

In [ ]:
d_time, d_dim = np.meshgrid(range(len(log['mode'])), range(7), indexing='ij')
plt.scatter(d_time.flatten(), d_dim.flatten(), c=log['mode'].flatten(), cmap='viridis', marker='s')
plt.show()

In [ ]:
actuator_config.GetControlLoopParameters(
    ActuatorConfig_pb2.LoopSelection(loop_selection=ActuatorConfig_pb2.JOINT_POSITION))

In [ ]:
router_options = RouterClientSendOptions()
router_options.timeout_ms = 1000
sessionManager.CloseSession(router_options)
transport.disconnect()

In [ ]:
log = collections.defaultdict(list)
actuator_mode = [ActuatorConfig_pb2.POSITION] * 7

base.SetServoingMode(Base_pb2.ServoingModeInformation(servoing_mode=Base_pb2.LOW_LEVEL_SERVOING))
try:
    feedback = base_cyclic.RefreshFeedback()
    out = ruckig.OutputParameter(7)
    res = ruckig.Result.Working
    start = time.monotonic()
    count = 0
    while res == ruckig.Result.Working:
        cur_pos = np.array([a.position for a in feedback.actuators])
        cur_vel = np.array([a.velocity for a in feedback.actuators])
        # inp_ctrl.current_position = cur_pos
        # inp_ctrl.current_velocity = cur_vel
        inp_ctrl.target_position = normalise_angles(target_joints, inp_ctrl.current_position)

        res = control.update(inp_ctrl, out)
        log['pos/current'].append(normalise_angles(cur_pos, inp_ctrl.current_position))
        log['pos/target'].append(inp_ctrl.target_position)
        log['vel/current'].append(cur_vel)
        log['pos/output'].append(out.new_position)
        log['vel/output'].append(out.new_velocity)
        out.pass_to_input(inp_ctrl)

        command.frame_id = (command.frame_id + 1) % 65536
        for i in range(7):
            command.actuators[i].command_id = command.frame_id
            # If the difference between the target and current position is less than 0.3 degrees,
            # use position control. Otherwise, use velocity control
            pos_control = abs(out.new_position[i] - inp_ctrl.target_position[i]) < 0.3 or abs(out.new_velocity[i]) < 0.1
            # mode = ActuatorConfig_pb2.POSITION if pos_control else ActuatorConfig_pb2.VELOCITY
            mode = ActuatorConfig_pb2.POSITION
            if actuator_mode[i] != mode:
                actuator_config.SetControlMode(ActuatorConfig_pb2.ControlModeInformation(control_mode=mode), deviceId=i+1)
                actuator_mode[i] = mode

            command.actuators[i].position = out.new_position[i]
            command.actuators[i].velocity = out.new_velocity[i]

        log['mode'].append([actuator_mode[i] for i in range(7)])
        feedback = base_cyclic.Refresh(command, 0, send_option)
        count += 1
        if count % 100 == 99:
            print(f'Running {count / (time.monotonic() - start):.2f} hz')

finally:
    for i in range(7):
        actuator_config.SetControlMode(
            ActuatorConfig_pb2.ControlModeInformation(control_mode=ActuatorConfig_pb2.POSITION),
            deviceId=i+1)
    # command.frame_id = (command.frame_id + 1) % 65536
    # for i in range(7):
    #     command.actuators[i].command_id = command.frame_id
    #     command.actuators[i].position = feedback.actuators[i].position
    # feedback = base_cyclic.Refresh(command, 0, send_option)

    base.SetServoingMode(Base_pb2.ServoingModeInformation(servoing_mode=Base_pb2.SINGLE_LEVEL_SERVOING))